#Semantic Role Labeling with ChatGPT
Copyright 2023 Denis Rothman, MIT License

This notebook shows how to use GPT-3.5(ChatGPT) with the OpenAI API to perform SRL tasks:

1. Installing openai and your API key<br>
2. Semantic Role Labeling as exaplained in *Tranformers for NLP, 2nd Edition, Chapter 10, Semantic Role Labeling.*

To get the best out of this notebook:
*   read Transformers for NLP, *Chapter 7, The Rise of Suprahuman Transformers with GPT-3 Engines* to understand the architecture of GPT engines. ChatGPT is GPT-3.5 turbo, a GPT model trained a large dataset. A GPT-3.5 turbo engine is a variant of a GPT Model.

*  go through the notebooks in the [BONUS directory](https://github.com/Denis2054/Transformers-for-NLP-2nd-Edition/tree/main/Bonus#readme) to learn how to implement GPT-3.5 turbo(ChatGPT).

*  once you have understood the theory of GPT transformers and explored the BONUS notebooks, try different sentences you find on social media that you would like to analyze in this notebook.




# December 6,2023 OpenAI API update

[This notebook has been updated. See README "Getting Started with OpenAI API" section before running this notebook](https://github.com/Denis2054/Transformers-for-NLP-2nd-Edition/blob/main/README.md#getting-started-with-openai-api)


In [1]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [2]:
# December 4,2023 update : Tiktoken required to install OpenAI on Google Colab
# Tiktoken is a fast BPE tokenizer
!pip install tiktoken

# December 4,2023 update : Cohere required to install OpenAI to implement language AI.
# Cohere platform: https://dashboard.cohere.com/
!pip install --upgrade cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires openai, which is not installed.


#1.Installing openai


## installing and importing openai

In [3]:
#Importing openai
try:
  import openai
except:
  !pip install openai
  import openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.1 MB/s eta 0:00:00


##API Key

In [4]:
#2.API Key
#Store you key in a file and read it(you can type it directly in the notebook but it will be visible for somebody next to you)
from google.colab import drive
drive.mount('/content/drive')
f = open("drive/MyDrive/files/api_key.txt", "r")
API_KEY=f.readline()
f.close()

#The OpenAI Key
import os
os.environ['OPENAI_API_KEY'] =API_KEY
openai.api_key = os.getenv("OPENAI_API_KEY")


Mounted at /content/drive


In [5]:
# December 6,2023
from openai import OpenAI
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

#2.ChatGPT dialog function

preparing the NLP message

In [6]:
 def dialog(uinput):
   #preparing the prompt for OpenAI
   role="user"

   #prompt="Where is Tahiti located?" #maintenance or if you do not want to use a microphone
   line = {"role": role, "content": uinput}

   #creating the mesage
   assert1={"role": "system", "content": "You are a Natural Language Processing Assistant."}
   assert2={"role": "assistant", "content": "You are helping viewers analyze social medial better."}
   assert3=line
   iprompt = []
   iprompt.append(assert1)
   iprompt.append(assert2)
   iprompt.append(assert3)

   #sending the message to ChatGPT
   response = client.chat.completions.create(model="gpt-4",messages=iprompt) #ChatGPT dialog
   text=response.choices[0].message.content

   return text

# 5.Semantic Role Labeling(SRL)

The 3 **basic** text samples are relatively easy. <br>
The 4 **difficult** text samples are challenging.

**Basic**

Sample1:
"Did Bob really think he could prepare a meal for 50 people in only a few hours?"

Sample 2:
"Mrs. and Mr. Tomaso went to Europe for vacation and visited Paris and first went to visit the Eiffel Tower."

Sample 3:
"John wanted to drink tea, Mary likes to drink coffee but Karim drank some cool water and Faiza would like to drink tomato juice."

**Difficult**

Sample 4 :ambiguous subject<br>
"Alice, whose husband went jogging every Sunday, liked to go to a dancing class in the meantime."

Sample 5: *round* can be a verb, a noun, preposition, etc.In this sentence, it is a verb<br>
"The bright sun, the blue sky, the warm sand, the palm trees, everything round off."

Sample 6: *ice* can be a noun or verb in Hockey<br>
"Now, ice pucks guys!"

Sample 7:confusing sentence with mixed descriptions<br>
"Though the customer seemed unhappy, she was, in fact satisfied but thinking of something else at the time, which gave a false impression?""

##Semantic Role Labeling(SRL) label list provided by ChatGPT

In [7]:
uinput="Provide the list of labels for Semantic Role Labeling"
text=dialog(uinput) #preparing the messages for ChatGPT
print("Viewer request",uinput)
print("ChatGPT Sentiment Analysis:",text)

Viewer request Provide the list of labels for Semantic Role Labeling
ChatGPT Sentiment Analysis: Semantic Role Labeling (SRL) aims at understanding the predicate-argument structure of a sentence. Here are some typical labels used in SRL:

1. Agent: The one who performs the action.
2. Patient: The entity that the action is performed upon.
3. Instrument: The object or the entities with the help of which the action is performed.
4. Location: The place where the action is performed.
5. Destination: The endpoint of the action or movement.
6. Source: The starting point of the action or movement.
7. Beneficiary: The one who benefits from the action.
8. Manner: It describes how the action is performed.
9. Purpose: It describes the reason or purpose behind the action.
10. Cause: The reason why the action happened.
11. Time: When the action is performed.
12. Theme: The entity that is moved or transformed.
13. Experiencer: The one who perceives or feels.

Remember, different systems may use diffe

## Basic samples

Sample1:<br>"Did Bob really think he could prepare a meal for 50 people in only a few hours?"

In [8]:
uinput="Perform Semantic Role Labeling on the following sentence:Did Bob really think he could prepare a meal for 50 people in only a few hours?"
text=dialog(uinput) #preparing the messages for ChatGPT
print("Viewer request",uinput)
print("ChatGPT Sentiment Analysis:",text)

Viewer request Perform Semantic Role Labeling on the following sentence:Did Bob really think he could prepare a meal for 50 people in only a few hours?
ChatGPT Sentiment Analysis: In Semantic Role Labeling (SRL), we try to identify the verbs in a sentence and then link other words or phrases to these verbs based on their role. In the given sentence, "think" and "prepare" are the two verbs.

For the verb "think":
1. Arguer (ARG0): Bob
2. Proposition (ARG1): he could prepare a meal for 50 people in only a few hours

For the verb "prepare":
1. Agent (ARG0): He (referring to Bob)
2. Beneficiary (ARG1): a meal for 50 people
3. Temporal (AM-TMP): in only a few hours

It's important to note that different SRL models might label these roles using different conventions. For instance, some might label "Bob" as ARG0 or ARG1 based on different interpretations.


Sample 2:<br>"Mrs. and Mr. Tomaso went to Europe for vacation and visited Paris and first went to visit the Eiffel Tower."

In [9]:
uinput="Perform Semantic Role Labeling on the following sentence:Mrs. and Mr. Tomaso went to Europe for vacation and visited Paris and first went to visit the Eiffel Tower."
text=dialog(uinput) #preparing the messages for ChatGPT
print("Viewer request",uinput)
print("ChatGPT Sentiment Analysis:",text)

Viewer request Perform Semantic Role Labeling on the following sentence:Mrs. and Mr. Tomaso went to Europe for vacation and visited Paris and first went to visit the Eiffel Tower.
ChatGPT Sentiment Analysis: In Semantic Role Labeling (SRL), the aim is to assign labels to words or phrases in a sentence that indicate their semantic role in the sentence. The semantic roles we usually look for are: Agent, Patient, Theme, Instrument, Location, Time, Cause, etc. 

Here's how these roles could be applied to the sentence "Mrs. and Mr. Tomaso went to Europe for vacation and visited Paris and first went to visit the Eiffel Tower.":

- Mrs. and Mr. Tomaso (Agent) went (Predicate) to Europe (Goal) for a vacation (Purpose). 

- They (Agent) visited (Predicate) Paris (Theme).

- They (Agent) first went to visit (Predicate) the Eiffel Tower (Theme).


Sample 3: <br>
"John wanted to drink tea, Mary likes to drink coffee but Karim drank some cool water and Faiza would like to drink tomato juice."

In [10]:
uinput="Perform Semantic Role Labeling on the following sentence:John wanted to drink tea, Mary likes to drink coffee but Karim drank some cool water and Faiza would like to drink tomato juice."
text=dialog(uinput) #preparing the messages for ChatGPT
print("Viewer request",uinput)
print("ChatGPT Sentiment Analysis:",text)

Viewer request Perform Semantic Role Labeling on the following sentence:John wanted to drink tea, Mary likes to drink coffee but Karim drank some cool water and Faiza would like to drink tomato juice.
ChatGPT Sentiment Analysis: Here is the semantic role labeling for the sentence:

Sentence: "John wanted to drink tea, Mary likes to drink coffee but Karim drank some cool water and Faiza would like to drink tomato juice."

1. John wanted to drink tea
   - Agent: John
   - Verb: wanted
   - Action: to drink
   - Object: tea

2. Mary likes to drink coffee
   - Agent: Mary
   - Verb: likes
   - Action: to drink
   - Object: coffee

3. Karim drank some cool water
   - Agent: Karim
   - Verb: drank
   - Object: some cool water

4. Faiza would like to drink tomato juice
   - Agent: Faiza
   - Mode: would like
   - Action: to drink
   - Object: tomato juice

The semantic roles - agent, verb and object - show who is doing what to whom. The verb or action is what is being performed by the agent a

## Difficult samples

Sample 4 :ambiguous subject<br>
"Alice, whose husband went jogging every Sunday, liked to go to a dancing class in the meantime."

In [11]:
uinput="Perform Semantic Role Labeling on the following sentence:Alice, whose husband went jogging every Sunday, liked to go to a dancing class in the meantime."
text=dialog(uinput) #preparing the messages for ChatGPT
print("Viewer request",uinput)
print("ChatGPT Sentiment Analysis:",text)

Viewer request Perform Semantic Role Labeling on the following sentence:Alice, whose husband went jogging every Sunday, liked to go to a dancing class in the meantime.
ChatGPT Sentiment Analysis: Semantic Role Labeling (SRL) aims to give a computational description of the predicate-argument structure of a sentence, and it associates semantic roles to the different constituents of the sentence. Here's the SRL for your sentence:

Sentence: Alice, whose husband went jogging every Sunday, liked to go to a dancing class in the meantime.

- Alice: Arguer (the one performing the action)
- Liked to go: Predicate (the main action or state described in the sentence)
- Whose husband went jogging every Sunday: Relative clause providing additional information about 'Alice'
  -- Husband: Arguer in the inner clause
  -- Went jogging: Predicate of the inner clause
  -- Every Sunday: Temporal argument of the inner clause
- To a dancing class: Arg2 (the destination of the action)
- In the meantime: Temp

Sample 5: *round* can be a verb, a noun, preposition, etc.In this sentence, it is a verb<br>
"The bright sun, the blue sky, the warm sand, the palm trees, everything round off."

In [12]:
uinput="Perform Semantic Role Labeling on the following sentence:The bright sun, the blue sky, the warm sand, the palm trees, everything round off."
text=dialog(uinput) #preparing the messages for ChatGPT
print("Viewer request",uinput)
print("ChatGPT Sentiment Analysis:",text)

Viewer request Perform Semantic Role Labeling on the following sentence:The bright sun, the blue sky, the warm sand, the palm trees, everything round off.
ChatGPT Sentiment Analysis: Semantic Role Labeling (SRL) deals with understanding the predicate-argument structure of a sentence and provides a representation (semantic roles) that answer: who did what to whom, what happened to who, and what is what. 

Here is the SRL of the sentence: 

"The bright sun, the blue sky, the warm sand, the palm trees, everything round off."

Now let's break this down:

1. "The bright sun, the blue sky, the warm sand, the palm trees, everything" - This phrase can be considered as 'Agent'. It refers to "who" or "what" performs the action.

2. "round off" - This is the 'Predicate' that refers to the action performed by the agent.

Please note: This sentence is a bit abstract. "Round off" is usually used in the context of numbers, meaning "to simplify a number by making it a little bit larger or smaller". Ho

Sample 6: *ice* can be a noun or verb in Hockey<br>
This sentence seems to difficult for ChatGPT. The model isn't good at neing hockey coach!<br>
"Now, ice pucks guys!"

In [13]:
uinput="Perform Semantic Role Labeling on the following sentence:Now, ice pucks guys!"
text=dialog(uinput) #preparing the messages for ChatGPT
print("Viewer request",uinput)
print("ChatGPT Sentiment Analysis:",text)

Viewer request Perform Semantic Role Labeling on the following sentence:Now, ice pucks guys!
ChatGPT Sentiment Analysis: Semantic Role Labeling (SRL) involves the process of identifying the predicate-argument structure of a sentence, and is often explained in terms of traditional school grammar.

In your sentence "Now, ice pucks guys!", it is a bit ambiguous without clear context and it may lead to different analysis. But let's try to break it down in two possibilities:

Possibility 1:
- The verb or predicate of this sentence could be "ice pucks" if "ice pucks" is being used as a verb phrase (though this would be unusual). 
- "guys" could then be the argument, or the entity that the action is being performed on.
This kind of interpretation would imply that "guys" are being told to "ice pucks" right now.

Possibility 2:
- "Ice pucks" could be considered a noun phrase, and "guys" could be someone to whom the speaker is addressing.
- There's no clear verb or predicate in this case.
This i

Sample 7:confusing sentence with mixed descriptions<br>
"Though the customer seemed unhappy, she was, in fact satisfied but thinking of something else at the time, which gave a false impression?"

In [14]:
uinput="Perform Semantic Role Labeling on the following sentence:Though the customer seemed unhappy, she was, in fact satisfied but thinking of something else at the time, which gave a false impression?"
text=dialog(uinput) #preparing the messages for ChatGPT
print("Viewer request",uinput)
print("ChatGPT Sentiment Analysis:",text)

Viewer request Perform Semantic Role Labeling on the following sentence:Though the customer seemed unhappy, she was, in fact satisfied but thinking of something else at the time, which gave a false impression?
ChatGPT Sentiment Analysis: Semantic Role Labeling (SRL) is a process that assigns labels to words or phrases in a sentence that indicate semantic roles such as agent, goal, etc. Here's the SRL for your sentence:

"Though the customer seemed unhappy, she was, in fact satisfied but thinking of something else at the time, which gave a false impression."

- Arg0 (Agent): the customer
- Predicate: seemed
- Arg1 (State): unhappy

In the second part of the sentence:

- Arg0 (Agent): she (refers back to the customer)
- Predicate: was
- Arg1 (State): satisfied

In the third part of the sentence:

- Arg0 (Agent): she (refers back to the customer)
- Predicate: thinking
- Arg1: of something else
- ArgM-TMP (Temporal): at the time 

In the final part of the sentence:

- Arg0 (Cause): which (